In [1]:
%load_ext autoreload
%autoreload 2
import os

import torch
from torch.utils.data import DataLoader
from datasets import TransformedRoboEireanData
import utils
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from models import MultiClassJetNet
torch.manual_seed(2)

In [2]:
# Default scalings for the default boxes determined by k-means clustering
default_box_scalings = torch.tensor(
    [
        [0.06549374, 0.12928654],
        [0.11965626, 0.26605093],
        [0.20708716, 0.38876095],
        [0.31018215, 0.47485098],
        [0.415882, 0.8048184],
        [0.7293086, 0.8216225],
    ]
)
encoder = utils.Encoder(default_box_scalings, ["robot"])
transformed_train_data = TransformedRoboEireanData(
    os.path.join("data", "transformed", "train"), encoder
)
transformed_val_data = TransformedRoboEireanData(
    os.path.join(
        "data",
        "transformed",
        "val",
    ),
    encoder,
)


In [3]:
learning_rate = 1e-3
train_loader = DataLoader(
    transformed_train_data, batch_size=32, shuffle=True, num_workers=0
)
val_loader = DataLoader(
    transformed_val_data, batch_size=32, shuffle=False, num_workers=0
)

pl_model = MultiClassJetNet(encoder, learning_rate)
trainer = pl.Trainer(
    limit_predict_batches=100, max_epochs=200, callbacks=[RichProgressBar()]
)
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy            │ MulticlassAccuracy │      0 │
│ 1 │ input_layer         │ NormConv2dReLU     │    146 │
│ 2 │ depth_wise_backbone │ Sequential         │  4.9 K │
│ 3 │ classifier          │ Sequential         │ 20.9 K │
│ 4 │ output_layer        │ Conv2d             │    900 │
└───┴─────────────────────┴────────────────────┴────────┘

Trainable params: 26.8 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 26.8 K                                                                                               
Total estimated model params size (MB): 0

Output()

/home/jonathan/hulks/ml/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:4
30: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/jonathan/hulks/ml/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:4
30: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(